UE20CS332 : Algorithms For Intelligence Web And Information Retrieval 

SRN : PES1UG20CS717

Name : Shal Ritvik Sinha

Section : L

In [126]:
import nltk
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import emot
from nltk.tokenize import word_tokenize
import string
import num2words
import nums_from_string
import inflect
from collections import defaultdict

In [127]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shalritvik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shalritvik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shalritvik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Shalritvik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [128]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.SnowballStemmer('english')
lemmatizer = nltk.stem.WordNetLemmatizer()


In [129]:
df = pd.read_csv('text_emotion.csv')
print(df.shape)
df.head()

(40000, 4)


,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [130]:
df = df.iloc[:1000]

In [131]:
df = pd.DataFrame(df['content'])

In [132]:
df.head()

,content
0,@tiffanylue i know i was listenin to bad habi...
1,Layin n bed with a headache ughhhh...waitin o...
2,Funeral ceremony...gloomy friday...
3,wants to hang out with friends SOON!
4,@dannycastillo We want to trade with someone w...


In [133]:
corpus = df["content"].values

A.Tokenization


In [134]:
def tokenize_corpus(content):
    return list(map(word_tokenize, content))

In [135]:
tokenized_corpus = tokenize_corpus(corpus)

In [136]:
for i in range(5):
    print(f"Tweet {i}: {tokenized_corpus[i]}")

Tweet 0: ['@', 'tiffanylue', 'i', 'know', 'i', 'was', 'listenin', 'to', 'bad', 'habit', 'earlier', 'and', 'i', 'started', 'freakin', 'at', 'his', 'part', '=', '[']
Tweet 1: ['Layin', 'n', 'bed', 'with', 'a', 'headache', 'ughhhh', '...', 'waitin', 'on', 'your', 'call', '...']
Tweet 2: ['Funeral', 'ceremony', '...', 'gloomy', 'friday', '...']
Tweet 3: ['wants', 'to', 'hang', 'out', 'with', 'friends', 'SOON', '!']
Tweet 4: ['@', 'dannycastillo', 'We', 'want', 'to', 'trade', 'with', 'someone', 'who', 'has', 'Houston', 'tickets', ',', 'but', 'no', 'one', 'will', '.']


E.Stopword removal

In [137]:
def remove_stopwords(tokenized_tweets):
    _function = lambda tweet: [word for word in tweet if word not in stopwords]
    return list(map(_function, tokenized_tweets))

tokenized_nostopw_corpus = remove_stopwords(tokenized_corpus)
for i in range(5):
    print(f"Tweet {i}: {tokenized_nostopw_corpus[i]}")

Tweet 0: ['@', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '=', '[']
Tweet 1: ['Layin', 'n', 'bed', 'headache', 'ughhhh', '...', 'waitin', 'call', '...']
Tweet 2: ['Funeral', 'ceremony', '...', 'gloomy', 'friday', '...']
Tweet 3: ['wants', 'hang', 'friends', 'SOON', '!']
Tweet 4: ['@', 'dannycastillo', 'We', 'want', 'trade', 'someone', 'Houston', 'tickets', ',', 'one', '.']


B.Case Folding

In [138]:
def case_folding(tokenized_tweets):
    _function = lambda tweet: [word.lower() for word in tweet]
    return list(map(_function, tokenized_tweets))
tokenized_nostopw_case_corpus = case_folding(tokenized_nostopw_corpus)
for i in range(5):
    print(f"Tweet {i}: {tokenized_nostopw_case_corpus[i]}")

Tweet 0: ['@', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '=', '[']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '...', 'waitin', 'call', '...']
Tweet 2: ['funeral', 'ceremony', '...', 'gloomy', 'friday', '...']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '!']
Tweet 4: ['@', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', ',', 'one', '.']


c. Removal of punctuation marks, emoticons, HTML tags and links

Removing Punctuation

In [139]:
def remove_punc(tokenized_tweets):
    _function = lambda tweet: [re.sub(r'[^\w\s]', '', word) for word in tweet]
    return list(map(_function, tokenized_tweets))
removed_punc = remove_punc(tokenized_nostopw_case_corpus)
for i in range(5):
    print(f"Tweet {i}: {removed_punc[i]}")

Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', '', 'one', '']


Removing Links

In [140]:
def remove_links(tokenized_tweets):
    _function = lambda tweet: [re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ',word) for word in tweet]
    return list(map(_function, tokenized_tweets))
removed_links= remove_links(removed_punc)
for i in range(5):
    print(f"Tweet {i}: {removed_links[i]}")
    

Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', '', 'one', '']


Removing HTML tags

In [141]:
def remove_html(tokenized_tweets):
    _function = lambda tweet: [re.sub('<.*?>', '', word) for word in tweet]
    return list(map(_function, tokenized_tweets))

removed_html = remove_html(removed_punc)

for i in range(5):
    print(f"Tweet {i}: {removed_html[i]}")

Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', '', 'one', '']


Removing Emojis

In [142]:
def remove_emoji(tokenized_tweets):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    _function = lambda tweet: [re.sub(emoj, '', word) for word in tweet]
    return list(map(_function, tokenized_tweets))
removed_emoji = remove_emoji(removed_html)

for i in range(5):
    print(f"Tweet {i}: {removed_emoji[i]}")


Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', '', 'one', '']


D. Convert numerical values to text (Ex: 10 -> “ten”)


In [143]:
def convert_to_words(tokenized_tweets):  
    p = inflect.engine()
    _function = lambda tweet: [inflect.engine().number_to_words(int(word)) if word.isdigit() else word for word in tweet]
    return list(map(_function, tokenized_tweets)) 
converted_to_words = convert_to_words(removed_emoji)
for i in range(5):
    print(f"Tweet {i}: {converted_to_words[i]}")


Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['wants', 'hang', 'friends', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'tickets', '', 'one', '']


G.Lemmatizing Words

In [144]:
def lemmatize_words(tokenized_tweets):
    _function = lambda tweet: [lemmatizer.lemmatize(word) for word in tweet]
    return list(map(_function, tokenized_tweets))
tokenized_nostopw_case_lemm_corpus = lemmatize_words(converted_to_words)
for i in range(5):
    print(f"Tweet {i}: {tokenized_nostopw_case_lemm_corpus[i]}")

Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['want', 'hang', 'friend', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'ticket', '', 'one', '']


F. Stemming

In [145]:
def stem_words(tokenized_tweets):
    _function = lambda tweet: [stemmer.stem(word) for word in tweet]
    return list(map(_function, tokenized_tweets))

tokenized_nostopw_case_stem_corpus = lemmatize_words(tokenized_nostopw_case_lemm_corpus)
for i in range(5):
    print(f"Tweet {i}: {tokenized_nostopw_case_stem_corpus[i]}")

Tweet 0: ['', 'tiffanylue', 'know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part', '', '']
Tweet 1: ['layin', 'n', 'bed', 'headache', 'ughhhh', '', 'waitin', 'call', '']
Tweet 2: ['funeral', 'ceremony', '', 'gloomy', 'friday', '']
Tweet 3: ['want', 'hang', 'friend', 'soon', '']
Tweet 4: ['', 'dannycastillo', 'we', 'want', 'trade', 'someone', 'houston', 'ticket', '', 'one', '']


Create a new column in the dataframe, called ‘docID’. The docID is a unique
serial number (starting from 1) that can identify a document in a collection

In [146]:
df['docID'] = range(1, len(df) + 1)
df.head()

,content,docID
0,@tiffanylue i know i was listenin to bad habi...,1
1,Layin n bed with a headache ughhhh...waitin o...,2
2,Funeral ceremony...gloomy friday...,3
3,wants to hang out with friends SOON!,4
4,@dannycastillo We want to trade with someone w...,5


Create Inverted index

In [147]:
def prepare_inverted_index(tokenized_tweets):
    inverted_index = defaultdict(list)
    for doc_id, document in enumerate(tokenized_tweets):
        for term in document:
            inverted_index[term].append(doc_id)
    return inverted_index

In [148]:
inverted_index = prepare_inverted_index(tokenized_nostopw_case_stem_corpus)
for term, postings_list in inverted_index.items():
    print(f"{term}: {postings_list}")

: [0, 0, 0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 8, 9, 9, 12, 12, 13, 16, 17, 17, 18, 19, 19, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 24, 24, 24, 24, 24, 24, 25, 26, 26, 26, 26, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 29, 29, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 36, 39, 39, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 42, 42, 43, 43, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 48, 48, 48, 48, 49, 49, 49, 49, 50, 51, 52, 53, 53, 53, 53, 54, 54, 54, 54, 56, 56, 56, 56, 56, 57, 57, 57, 58, 58, 59, 59, 59, 59, 60, 60, 60, 61, 61, 61, 62, 62, 62, 62, 63, 63, 63, 64, 64, 64, 64, 64, 64, 66, 66, 66, 67, 67, 68, 69, 69, 69, 69, 69, 69, 69, 70, 70, 71, 71, 71, 71, 71, 71, 72, 72, 73, 73, 74, 74, 74, 74, 75, 76, 76, 76, 77, 77, 77, 77, 78, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 80, 81, 81, 81, 82, 82, 82, 82, 83, 83, 83, 83, 83, 84

Extend the postings list prepared in the above step to make a positional postings list
and print the same.

In [149]:
def prepare_positional_inverted_index(tokenized_tweets):
    inverted_index = defaultdict(dict)
    for doc_id, document in enumerate(tokenized_tweets):
        for position, term in enumerate(document.split()):
            if doc_id not in inverted_index[term]:
                inverted_index[term][doc_id] = []
            inverted_index[term][doc_id].append(position)
    return inverted_index

In [150]:
positional_inverted_index = prepare_positional_inverted_index(inverted_index)

for term, postings_list in positional_inverted_index.items():
    print(f"{term}: {postings_list}")

tiffanylue: {1: [0]}
know: {2: [0]}
listenin: {3: [0]}
bad: {4: [0]}
habit: {5: [0]}
earlier: {6: [0]}
started: {7: [0]}
freakin: {8: [0]}
part: {9: [0]}
layin: {10: [0]}
n: {11: [0]}
bed: {12: [0]}
headache: {13: [0]}
ughhhh: {14: [0]}
waitin: {15: [0]}
call: {16: [0]}
funeral: {17: [0]}
ceremony: {18: [0]}
gloomy: {19: [0]}
friday: {20: [0]}
want: {21: [0]}
hang: {22: [0]}
friend: {23: [0]}
soon: {24: [0]}
dannycastillo: {25: [0]}
we: {26: [0]}
trade: {27: [0]}
someone: {28: [0]}
houston: {29: [0]}
ticket: {30: [0]}
one: {31: [0], 840: [0], 844: [0], 1122: [0], 1362: [0, 2]}
repinging: {32: [0]}
ghostridah14: {33: [0]}
nt: {34: [0]}
go: {35: [0]}
prom: {36: [0]}
bc: {37: [0]}
bf: {38: [0]}
like: {39: [0]}
i: {40: [0]}
sleep: {41: [0]}
im: {42: [0]}
thinking: {43: [0]}
old: {44: [0]}
s: {45: [0]}
married: {46: [0]}
damn: {47: [0]}
amp: {48: [0]}
two: {49: [0], 139: [0], 1226: [0], 1810: [0], 3140: [0, 5]}
scandalous: {50: [0]}
hmmm: {51: [0]}
http: {52: [0]}
wwwdjherocom: {53: [0]}
ch